<a href="https://colab.research.google.com/github/alinspiron/TF2.x-Object-Detection-Tutorial-on-Weapon-Dataset/blob/main/Weapon_Detection_TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Tensorflow 2.x Object Detection Tutorial on Weapon Dataset**

**Authors:** *Mr. Ali shafique, Zafar Iqbal, Aftab Karim, Ayesha Afzal, Bushra Hafeez*

Group under the supervision of Mr. Ali Shafique in the Department of Electrical Engineering at University of Engineering and Technology Lahore. In this blog Efficient-d0 model has been trained on weapon dataset.

In this tutorial we will learn how to train object detection model using TensorFlow 2 object detection API. TensorFlow 2 object detection API has been released on 10th July 2020 with lots of improvements as compared to the TensorFlow version 1. There has been a lot of new deep learning models with better computational speed and Mean Average Precision (mAP) on COCO Dataset. We will use Google Collaboratory to train the model, using the free GPU server and then export the trained model. In this tutorial we will use weapon dataset. This tutorial includes all the necessary code cells to train the model for object detection. We have got a great deal of help from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/. You can learn more about this notebook on medium article by following this link ___.

#1. Importing Libraries

In [ ]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv
import re
import cv2
import os
import glob
import xml.etree.ElementTree as ET
import argparse

import io
import tensorflow as tf

from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
2.4.0


#2. Mounting Google Drive (optional)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#3. Creating Directory Structure

In [ ]:
!mkdir Tensorflow

%cd /content/Tensorflow/

!mkdir annotations

!mkdir images

!mkdir test

!mkdir train

!mkdir models

!mkdir pre-trained-models

!mkdir my-model

!mkdir exported-model

/content/Tensorflow


#4. Downloading Dataset

In [ ]:
%cd '/content/Tensorflow/'

#download the images zip
!wget https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/WeaponS.zip

#unzip the image file
!unzip -q WeaponS.zip

#download the annotations zip
!wget https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/WeaponS_bbox.zip

#unzip the annotations file
!unzip -q WeaponS_bbox.zip


/content/Tensorflow
--2020-12-26 13:48:31--  https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/WeaponS.zip
Resolving sci2s.ugr.es (sci2s.ugr.es)... 150.214.190.154
Connecting to sci2s.ugr.es (sci2s.ugr.es)|150.214.190.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 250005059 (238M) [application/zip]
Saving to: ‘WeaponS.zip’

WeaponS.zip         100%[===================>] 238.42M  11.2MB/s    in 22s     

2020-12-26 13:48:53 (10.9 MB/s) - ‘WeaponS.zip’ saved [250005059/250005059]

--2020-12-26 13:48:55--  https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/WeaponS_bbox.zip
Resolving sci2s.ugr.es (sci2s.ugr.es)... 150.214.190.154
Connecting to sci2s.ugr.es (sci2s.ugr.es)|150.214.190.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1420022 (1.4M) [application/zip]
Saving to: ‘WeaponS_bbox.zip’

WeaponS_bbox.zip    100%[===================>]   1.35M

#5. Organizing Dataset

In [ ]:
%cd '/content/Tensorflow/'

# moves the IMAGES to images folder
!mv WeaponS/* '/content/Tensorflow/images/'

# Moves the Random 600 labels to the test dir
!ls /content/Tensorflow/WeaponS_bbox/* | sort -R | head -600 | xargs -I{} mv {} '/content/Tensorflow/test/'

# moves the remaining annotations to train folder
!mv WeaponS_bbox/* '/content/Tensorflow/train/'

#Check the contents of images folder
!ls -1 '/content/Tensorflow/images/' | wc -l

#Check the contents of test folder
!ls -1 '/content/Tensorflow/test/' | wc -l

#Check the contents of train folder
!ls -1 '/content/Tensorflow/train/' | wc -l

/content/Tensorflow
3000
600
2400


In [ ]:
%cd '/content/Tensorflow/'

# Deleting the zipped, unzipped folders and empty folders 
!rm -rf WeaponS_bbox.zip

!rm -rf WeaponS.zip

!rm -rf WeaponS_bbox

!rm -rf WeaponS


/content/Tensorflow


#6. Creating CSV File

In [ ]:

%cd '/content/Tensorflow/'

# images extension
images_extension = 'jpg'

def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text + '.' + images_extension,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

# for both the train and test csv files, it runs the xml_to_csv() above.
for label_path in ['test', 'train']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')


/content/Tensorflow
Successfully converted test xml to csv.
Successfully converted train xml to csv.


#7. Creating Label Map

In [ ]:
# Creating the `label_map.pbtxt` file
label_map_path = os.path.join("label_map.pbtxt")

pbtxt_content = ""

#creats a pbtxt file the has the class names.
for i, class_name in enumerate(classes):
    # display_name is optional.
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n    display_name: 'Gun'\n }}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

#Copy the label_map to annotations folder
!cp label_map.pbtxt  '/content/Tensorflow/annotations/'

#checking the context of pbtxt file
!cat label_map.pbtxt


item {
    id: 1
    name: 'pistol'
    display_name: 'Gun'
 }

#8. Checking Errors in Dataset

In [ ]:
%cd '/content/Tensorflow/'
# path to images
images_path = 'images'

for CSV_FILE in ['train.csv', 'test.csv']:
  with open(CSV_FILE, 'r') as fid:  
      print('[*] Checking file:', CSV_FILE) 
      file = csv.reader(fid, delimiter=',')
      first = True 
      cnt = 0
      error_cnt = 0
      error = False
      for row in file:
          if error == True:
              error_cnt += 1
              error = False         
          if first == True:
              first = False
              continue     
          cnt += 1      
          name, width, height, xmin, ymin, xmax, ymax = row[0], int(row[1]), int(row[2]), int(row[4]), int(row[5]), int(row[6]), int(row[7])     
          path = os.path.join(images_path, name)
          img = cv2.imread(path)         
          if type(img) == type(None):
              error = True
              print('Could not read image', img)
              continue     
          org_height, org_width = img.shape[:2]     
          if org_width != width:
              error = True
              print('Width mismatch for image: ', name, width, '!=', org_width)     
          if org_height != height:
              error = True
              print('Height mismatch for image: ', name, height, '!=', org_height) 
          if xmin > org_width:
              error = True
              print('XMIN > org_width for file', name)  
          if xmax > org_width:
              error = True
              print('XMAX > org_width for file', name)
          if ymin > org_height:
              error = True
              print('YMIN > org_height for file', name)
          if ymax > org_height:
              error = True
              print('YMAX > org_height for file', name)
          if error == True:
              print('Error for file: %s' % name)
              print()
      print()
      print('Checked %d files and realized %d errors' % (cnt, error_cnt))
      print("-----")


/content/Tensorflow
[*] Checking file: train.csv

Checked 2776 files and realized 0 errors
-----
[*] Checking file: test.csv
XMIN > org_width for file armas (2815).jpg
XMAX > org_width for file armas (2815).jpg
YMIN > org_height for file armas (2815).jpg
YMAX > org_height for file armas (2815).jpg
Error for file: armas (2815).jpg


Checked 688 files and realized 1 errors
-----


In [ ]:
#removing the image 
!rm /content/Tensorflow/images/'armas (2815).jpg'


#removing the entry for it in the csv for that image as well

#because we did a random split for the data, we dont know if it ended up being in training or testing
# we will remove the image from both.

#train_Set
#reading the training csv
df = pd.read_csv('/content/Tensorflow/train.csv')
# removing armas (2815).jpg
df = df[df['filename'] != 'armas (2815).jpg']
#reseting the index
df.reset_index(drop=True, inplace=True)
#saving the df
df.to_csv('/content/Tensorflow/train.csv')

#test_Set
#reading the testing csv
df = pd.read_csv('/content/Tensorflow/test.csv')
# removing armas (2815).jpg
df = df[df['filename'] != 'armas (2815).jpg']
#reseting the index
df.reset_index(drop=True, inplace=True)
#saving the df
df.to_csv('/content/Tensorflow/test.csv')

# Just for the memory
df = None


#9. Downloading Tensorflow Model Garden

In [ ]:
#cd into the TensorFlow directory in your Google Drive
%cd '/content/Tensorflow/'
#and clone the TensorFlow Model Garden repository
!git clone https://github.com/tensorflow/models.git


/content/Tensorflow
Cloning into 'models'...
remote: Enumerating objects: 49553, done.
remote: Total 49553 (delta 0), reused 0 (delta 0), pack-reused 49553
Receiving objects: 100% (49553/49553), 558.64 MiB | 38.94 MiB/s, done.
Resolving deltas: 100% (34183/34183), done.


#10. Installing and Compiling Required Packages

In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim
!pip install lvis

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,615 kB of archives.
After this operation, 8,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-pkg-resources all 39.0.1-2 [128 kB]
Get:3 http:/

In [ ]:
#cd into 'TensorFlow/models/research'
%cd '/content/Tensorflow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.


/content/Tensorflow/models/research


#11. Setting up Training Environment

In [ ]:
import os
import sys

os.environ['PYTHONPATH']+=":/content/Tensorflow/models"
sys.path.append("/content/Tensorflow/models/research")


In [ ]:
!python3 setup.py build
!python3 setup.py install


running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/exporter_lib_v2.py -> build/lib/object_detection
copying object_detection/model_main_tf2.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/exporter_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/model_lib.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_gra

#12. Testing the Installation

In [ ]:
#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/Tensorflow/models/research/object_detection/builders/'

!python3 model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')


/content/Tensorflow/models/research/object_detection/builders
2020-12-26 13:56:47.067207: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-12-26 13:56:49.842593: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-26 13:56:49.843573: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-26 13:56:49.848038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-26 13:56:49.848571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
cor

#13. Generating TF Records

In [ ]:
#adjusted from: https://github.com/datitran/raccoon_dataset
from object_detection.utils import dataset_util

%cd '/content/Tensorflow/'

#change this to the base directory where your data/ is 
data_base_url = '/content/Tensorflow/'

#location of images
image_dir = data_base_url + 'images/'

def class_text_to_int(row_label):
  if row_label == 'pistol':
    return 1
  else:
    None

def split(df, group):
  data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
  with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = Image.open(encoded_jpg_io)
  width, height = image.size
  filename = group.filename.encode('utf8')
  image_format = b'jpg'
  xmins = []
  xmaxs = []
  ymins = []
  ymaxs = []
  classes_text = []
  classes = []

  for index, row in group.object.iterrows():
    xmins.append(row['xmin'] / width)
    xmaxs.append(row['xmax'] / width)
    ymins.append(row['ymin'] / height)
    ymaxs.append(row['ymax'] / height)
    classes_text.append(row['class'].encode('utf8'))
    classes.append(class_text_to_int(row['class']))

  tf_example = tf.train.Example(features=tf.train.Features(feature={
    'image/height': dataset_util.int64_feature(height),
    'image/width': dataset_util.int64_feature(width),
    'image/filename': dataset_util.bytes_feature(filename),
    'image/source_id': dataset_util.bytes_feature(filename),
    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
    'image/format': dataset_util.bytes_feature(image_format),
    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
    'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
  return tf_example

#creates tfrecord for both csv's
for csv in ['train', 'test']:
  writer = tf.io.TFRecordWriter(data_base_url + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(data_base_url + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), data_base_url + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(data_base_url +csv + '.record'))



/content/Tensorflow
Successfully created the TFRecords: /content/Tensorflow/train.record
Successfully created the TFRecords: /content/Tensorflow/test.record


In [ ]:
%cd '/content/Tensorflow/'

!cp train.record  '/content/Tensorflow/annotations/'

!cp test.record  '/content/Tensorflow/annotations/'

/content/Tensorflow


#14. Downloading the Required Model for Training

Link to Tensorflow 2 object detection model zoo https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
%cd '/content/Tensorflow/pre-trained-models/' 


#Link to download efficientdet_d0
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz

!tar -xvf  efficientdet_d0_coco17_tpu-32.tar.gz


/content/Tensorflow/pre-trained-models
--2020-12-26 13:58:49--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.13.80, 2607:f8b0:4004:808::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.13.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M  --.-KB/s    in 0.1s    

2020-12-26 13:58:49 (240 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]

efficientdet_d0_coco17_tpu-32/
efficientdet_d0_coco17_tpu-32/checkpoint/
efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
efficientdet_d0_coco17_tpu-32/checkpoint/checkpoint
efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0.index
efficientdet_d0_coco17_tpu-32/pipeline.config
efficientdet_d0_co

In [ ]:
%cd '/content/Tensorflow/models/research/object_detection/'

!cp model_main_tf2.py '/content/Tensorflow/'

!cp exporter_main_v2.py '/content/Tensorflow/'

%cd '/content/Tensorflow/pre-trained-models/efficientdet_d0_coco17_tpu-32/'

!cp pipeline.config '/content/Tensorflow/my-model/'

%cd '/content/Tensorflow/my-model/'


/content/Tensorflow/models/research/object_detection
/content/Tensorflow/pre-trained-models/efficientdet_d0_coco17_tpu-32
/content/Tensorflow/my-model


#15. Configuring pipeline File

In [ ]:
%%writefile pipeline.config

model {
  ssd {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 512
        max_dimension: 512
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: "ssd_efficientnet-b0_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.9900000095367432
          scale: true
          epsilon: 0.0010000000474974513
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 3
        num_filters: 64
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 1.0
        x_scale: 1.0
        height_scale: 1.0
        width_scale: 1.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.9999998989515007e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: SWISH
          batch_norm {
            decay: 0.9900000095367432
            scale: true
            epsilon: 0.0010000000474974513
          }
          force_use_bias: true
        }
        depth: 64
        num_layers_before_predictor: 3
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        scales_per_octave: 3
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 9.99999993922529e-09
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 1.5
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    add_background_class: false
  }
}
train_config {
  batch_size: 4
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_scale_crop_and_pad_to_square {
      output_size: 512
      scale_min: 0.10000000149011612
      scale_max: 2.0
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.07999999821186066
          total_steps: 300000
          warmup_learning_rate: 0.0010000000474974513
          warmup_steps: 2500
        }
      }
      momentum_optimizer_value: 0.8999999761581421
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "/content/Tensorflow/pre-trained-models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0"
  num_steps: 30000
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: false
  fine_tune_checkpoint_version: V2
}
train_input_reader: {
  label_map_path: "/content/Tensorflow/annotations/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/Tensorflow/annotations/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/content/Tensorflow/annotations/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/Tensorflow/annotations/test.record"
  }
}


Overwriting pipeline.config


#16. Starting TensorBoard

In [ ]:
#cd into Tensorflow folder
%cd '/content/Tensorflow'

#start the Tensorboard
%load_ext tensorboard
%tensorboard --logdir='/content/Tensorflow/my-model/'


#17. Training 

In [ ]:
%cd '/content/Tensorflow/'

/content/Tensorflow


In [ ]:
!python3 model_main_tf2.py \
      --model_dir='/content/Tensorflow/my-model/' \
      --pipeline_config_path='/content/Tensorflow/my-model/pipeline.config' \
      --alsologtostderr


2020-12-26 14:05:13.389895: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-26 14:05:15.811701: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-26 14:05:15.812631: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-26 14:05:15.816943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-26 14:05:15.817479: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-12-26 14:05:15.817508: I tensorflow/stream_executor/platform/default/dso_loade

#18. Exporting the Model

In [ ]:
!python3 exporter_main_v2.py \
      --input_type image_tensor \
      --pipeline_config_path='/content/Tensorflow/my-model/pipeline.config' \
      --trained_checkpoint_dir='/content/Tensorflow/my-model/' \
      --output_directory='/content/Tensorflow/exported-model/'


#19. Loading the Saved Model

In [ ]:
#Loading the saved_model(change the path according to your directory names)
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="/content/Tensorflow/my-model/saved_model/"
print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')


#20. Loading Label_Map

In [ ]:
#Loading the label_map

category_index=label_map_util.create_category_index_from_labelmap("/content/Tensorflow/annotations/label_map.pbtxt",use_display_name=True)


#21. Running Inference

In [ ]:
#Loading the image
img=[ '/content/img2.jpg' ]

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

for image_path in img:
    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)

    input_tensor=tf.convert_to_tensor(image_np)
    input_tensor=input_tensor[tf.newaxis, ...]

    detections=detect_fn(input_tensor)
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
    for key,value in detections.items()}
    detections['num_detections']=num_detections
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)
    image_np_with_detections=image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=100,     
          min_score_thresh=.5,      
          agnostic_mode=False)

%matplotlib inline
plt.figure()
plt.imshow(image_np_with_detections)
print('Done')
plt.show()
